In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
from tqdm import tqdm

In [2]:
chemin_paradonnees = r"C:\Users\SHZ42C\Documents\RTI\autocompletion\EEC_hackathon.csv"

In [7]:
import time

In [17]:
driver = webdriver.Firefox()

In [18]:
driver.get("https://stromae-90-01z.dev.insee.io/questionnaire/90-01z/unite-enquetee/11")

In [19]:
driver.find_element(By.CSS_SELECTOR, ".undefined:nth-child(2) > .MuiButton-label").click()
driver.find_element(By.CSS_SELECTOR, ".MuiButton-label").click()
element = driver.find_element(By.CSS_SELECTOR, "#next-button > .MuiButton-label")
actions = ActionChains(driver)
actions.move_to_element(element).perform()
driver.find_element(By.CSS_SELECTOR, "#next-button > .MuiButton-label").click()
driver.find_element(By.CSS_SELECTOR, "#next-button > .MuiButton-label").click()


In [16]:
driver.quit()

In [45]:
def interroger_stromae(driver, libelle : str) -> list[str]:
    driver.find_element(By.ID, "l5smshnt-input").clear()
    driver.find_element(By.ID, "l5smshnt-input").send_keys(libelle)
    time.sleep(1)
    element_liste = driver.find_element(By.ID, "l5smshnt-list")
    return [x.text for x in element_liste.find_elements(By.CLASS_NAME, 'label') ]

In [32]:
interroger_stromae(driver,"prof")

["Professeur de lycée d'enseignement professionnel (LEP)",
 "Aide familial (d'une profession libérale)",
 'Basketteur professionnel',
 'Bouliste professionnel',
 'Boxeur professionnel',
 'Chargé de prévention des risques professionnels',
 'Chasseur (professionnel)',
 'Conseiller en évolution professionnelle',
 'Conseiller en insertion professionnelle',
 'Cycliste professionnel',
 'Footballeur professionnel',
 'Handballeur professionnel',
 'Head of professional services',
 'Hockeyeur professionnel',
 'Jockey professionnel',
 'Lutteur professionnel',
 'Néviplanchiste professionnel',
 'Planchiste professionnel',
 'Préventionniste en risques professionnels',
 'Professeur à domicile',
 'Professeur agrégé dans le supérieur (PRAG)',
 'Professeur auxiliaire',
 'Professeur certifié dans le supérieur (PRCE)',
 "Professeur d'art dramatique",
 "Professeur d'éducation artistique ou musicale",
 "Professeur d'éducation physique et sportive (EPS)",
 "Professeur d'enseignement général de collège (pegc)

In [33]:
def tester_presence_stromae(libelles_retournes: list[str], libelle_expected: str):
    return libelle_expected in libelles_retournes

In [48]:
data = pd.read_csv(chemin_paradonnees,dtype=str)
data = data[["PCLCA","PCLCALIBELLEMAX","SEXE"]]
data = data.loc[~data["PCLCA"].isna(),]
data = data.loc[data["PCLCA"]!="999",]
data = data.loc[data["SEXE"]=="1"]

In [49]:
data = data.head(100)

In [50]:
data

,PCLCA,PCLCALIBELLEMAX,SEXE
3,Dessinateur projeteur industriel,Dessinateur in,1
4,Contrôleur qualité de l'industrie,contrôleur qualité,1
5,Technicien d'assistance en informatique,tech,1
13,Vendeur d'articles de sport,VENDEUR,1
22,Apprenti maçon,apprenti macon,1
...,...,...,...
489,Agent de gestion des collectivités territoriales,AGENT TERRITORIALES,1
518,"Chauffeur de bus, de car",chauffeur,1
523,Ingénieur qualité,ingénieur qualité,1
532,Responsable des services techniques,RESPONSABLE T,1


In [52]:
def benchmark_stromae(data, driver):
    presence_stromae_list = []
    for i in tqdm(data.index):
        prof_sel = data.at[i,"PCLCA"]
        prof_input = data.at[i,"PCLCALIBELLEMAX"]
        propositions = interroger_stromae(driver=driver,libelle=prof_input)
        presence_stromae_list.append(tester_presence_stromae(libelle_expected=prof_sel,libelles_retournes=propositions))
    data.insert(loc=0,column="I_PRESENCE_STROMAE",value=presence_stromae_list)    

In [53]:
benchmark_stromae(data, driver)

100%|██████████| 100/100 [02:08<00:00,  1.29s/it]


In [54]:
data

,I_PRESENCE_STROMAE,PCLCA,PCLCALIBELLEMAX,SEXE
3,False,Dessinateur projeteur industriel,Dessinateur in,1
4,False,Contrôleur qualité de l'industrie,contrôleur qualité,1
5,False,Technicien d'assistance en informatique,tech,1
13,False,Vendeur d'articles de sport,VENDEUR,1
22,True,Apprenti maçon,apprenti macon,1
...,...,...,...,...
489,False,Agent de gestion des collectivités territoriales,AGENT TERRITORIALES,1
518,False,"Chauffeur de bus, de car",chauffeur,1
523,True,Ingénieur qualité,ingénieur qualité,1
532,False,Responsable des services techniques,RESPONSABLE T,1


In [55]:
sum(data.I_PRESENCE_STROMAE)

32